In [1]:
import numpy as np
import torch
import sys
from pathlib import Path
import importlib

import pandas as pd
import torchaudio

# module_path = str(Path.cwd().parents[0] / "utils")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# if module_path not in sys.path:
#     sys.path.append(module_path)
# module_path = str(Path.cwd().parents[0] / "network_models/w2v_emotion")
# if module_path not in sys.path:
#     sys.path.append(module_path)
# module_path = str(Path.cwd().parents[0] / "network_models/soundstream_lstm")
# if module_path not in sys.path:
#     sys.path.append(module_path)

import utils.wav2Vec_utils as w2vU
import utils.audio_dataset_utils as ADU
import network_models.soundstream_lstm.LSTM_dataset as lds
import network_models.soundstream_lstm.CombinedEmoDataset_7_Emo as ced
import  network_models.w2v_emotion_model.custom_collator as cc
import network_models.w2v_emotion_model.custom_model as cm
import network_models.w2v_emotion_model.model_trainer as ct
import gc

model_name_or_path = "facebook/wav2vec2-large-960h-lv60-self"
pooling_mode = "mean"
device = "cuda"
emo_dataset = ced.CombinedEmoDataSet_7_emos(directory_tess="/home/ckwdani/Music/emotionDatasets/converted_mono/tess",
                                            directory_cafe="/home/ckwdani/Music/emotionDatasets/converted_mono/cafe",
                                            directory_ravdess="/home/ckwdani/Music/emotionDatasets/converted_mono/RAVDESS Audio_Speech_Actors_01-24",
                                            directory_mesd="/home/ckwdani/Music/emotionDatasets/converted_mono/mesd",
                                            with_dataset=True,
                                            transFormAudio=ced.collateToSeconds(5, 16000, const_value=0, device=device),
                                            device=device)



#emo_dataset = lds.AudioEmotionTessDataset(directory="/home/ckwdani/Music/emotionDatasets/converted_mono/tess", device=device)



2023-03-10 09:09:43.677160: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-10 09:09:44.180664: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-10 09:09:44.180715: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-10 09:09:44.180720: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
processor, sr = w2vU.init_w2v2(num_labels=len(emo_dataset.label_list), label_list=emo_dataset.label_list, device=device)
newSet = lds.AudioEmotionTessWav2VecDataset(emo_dataset, processor= processor, sampling_rate=sr)
gc.collect()
modelEncodeModel = cm.Wav2Vec2ForSpeechClassification(model_name_or_path=model_name_or_path, pooling_mode="mean", device=device, num_emotions=len(emo_dataset.label_list), dataset_generator=True)
modelEncodeModel.freeze_feature_extractor()

In [3]:

from network_models.soundsream_models_and_utils.wav2Vec_encoded_dataset import W2V_EncDs

# data_collator = cc.DataCollatorCTCWithPadding(processor=processor, padding=True, num_labels=len(newSet.dataSet.label_list))
#
# data = W2V_EncDs(model=modelEncodeModel, preencoded_dataset=newSet, emo_dataset=emo_dataset, device=device, encode=True, data_colator=data_collator)
# data.saveToPkl("/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/datasets/w2vEnc", "/w2v_tess_3_5.pkl")
#data = W2V_EncDs(model=modelEncodeModel, emo_dataset=emo_dataset, device=device, encode=False)
#data.fromPkl("/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/datasets/w2vEnc", "/w2v_tess_3_5.pkl")


# trainDS, testDs = ADU.train_val_dataset(newSet, val_split=0.2, seed=100)
# valDs, testDs = ADU.train_val_dataset(testDs, val_split=0.5, seed=100)
#trainSet, evalSet = ADU.train_val_dataset(newSet, 0.1)
#trainSet, evalSet = ADU.train_val_dataset(evalSet, 0.1)

#model = cm.Wav2Vec2ForSpeechClassification(model_name_or_path=model_name_or_path, pooling_mode="mean", device=device, num_emotions=len(emo_dataset.label_list), from_full_dataset=True)



# trainer = ct.ModelTrainer(model=model, need_reshape=False, train_dataset=trainSet, eval_dataset=evalSet, device=device, batch_size=2, save_model_every=500, num_epochs=5001, model_path="../content/classifier/W2VClassifier/Nr1/", data_collator=data_collator)
# gc.collect()
# trainer.train()

In [4]:
#trainDS, testDs = ADU.train_val_dataset(data, val_split=0.2, seed=100)
#valDs, testDs = ADU.train_val_dataset(testDs, val_split=0.5, seed=100)

In [5]:
# #Path("/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks_clip/content/datasets/w2vEnc").mkdir(parents=True)
# #data.to_pickle("/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks_clip/content/datasets/w2vEnc/w2v_3_5.pkl")
# model = cm.Wav2Vec2ForSpeechClassification(model_name_or_path=model_name_or_path, pooling_mode="mean", device=device, num_emotions=len(emo_dataset.label_list), from_full_dataset=True)


In [8]:
importlib.reload(ct)
gc.collect()
torch.manual_seed(4215)
model = cm.Wav2Vec2ForSpeechClassification(model_name_or_path=model_name_or_path, pooling_mode="mean", device=device, num_emotions=len(emo_dataset.label_list))
model.freeze_feature_extractor()
# model.load_state_dict(torch.load("/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/classifier/W2VClassifier/final_SGD/emo_reco_300.pth"))
data_collator = cc.DataCollatorCTCWithPadding(processor=processor, padding=True, num_labels=len(newSet.dataSet.label_list))
trainDS, testDs = ADU.train_val_dataset(newSet, val_split=0.2, seed=100)
valDs, testDs = ADU.train_val_dataset(testDs, val_split=0.5, seed=100)
trainer = ct.ModelTrainer(model=model, need_reshape=False, train_dataset=trainDS, eval_dataset=valDs, device=device, batch_size=4, save_model_every=50, num_epochs=1001, model_path="../content/classifier/W2VClassifier/final_SGD2/", labelList=emo_dataset.label_list, data_collator=data_collator)
#gc.collect()
trainer.train()

Epoch 1
-------------------------------
loss: 1.953197  [    0/ 4873]
loss: 1.945875  [  400/ 4873]
loss: 1.945449  [  800/ 4873]
loss: 1.947213  [ 1200/ 4873]
loss: 1.943246  [ 1600/ 4873]
loss: 1.944570  [ 2000/ 4873]
loss: 1.940568  [ 2400/ 4873]
loss: 1.947700  [ 2800/ 4873]
loss: 1.940200  [ 3200/ 4873]
loss: 1.951981  [ 3600/ 4873]
loss: 1.943350  [ 4000/ 4873]
loss: 1.942116  [ 4400/ 4873]
loss: 1.935929  [ 4800/ 4873]
           accuracy  precision  recall   support
   angry     0.136     0.000     0.000    91
 disgust     0.136     0.000     0.000    108
    fear     0.136     0.000     0.000    74
   happy     0.136     0.137     1.000    83
 neutral     0.136     0.000     0.000    89
     sad     0.136     0.000     0.000    105
surprise     0.136     0.000     0.000    59
                                          609

 
     avg     0.136     0.020     0.143    

                   angry     disgust        fear       happy     neutral         sad    surprise    support
   

KeyboardInterrupt: 

In [10]:
torch.save(model.state_dict(), "../content/classifier/W2VClassifier/final_SGD2/" + f"emo_reco_acc_67.pth")

In [33]:
from network_models.soundsream_models_and_utils.ss_model_dim_red import SmallDimRed, SSDimRedModel
from network_models.soundsream_models_and_utils.ss_trainer_gen_models import SSGenModelTrainer


torch.manual_seed(33333)
model = SSDimRedModel(768, reduceDims=False).to(device)
batch_size = 6
models_dir = "/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/classifier/W2VClassifier/final_dr/"
#models_dir = "/home/ckwdani/Programming/Projects/masterarbeit/Jupyter/mainProject/notebooks/content/soundstream/experiments/3_5_sec_dimred/Nr1/"
epochs = 1000
save_highest_acc_min_acc = 0.5
save_model_every = 50
lr = 1e-5
lr_quotient = 2
trainer = SSGenModelTrainer(lr=lr, num_epochs=epochs, model=model, train_dataset=trainDS,
                            eval_dataset=valDs,
                            device=device, labelList=emo_dataset.label_list,
                            batch_size=batch_size,
                            save_model_every=save_model_every,
                            save_highest_acc_min_acc=save_highest_acc_min_acc,
                            model_path = models_dir)

In [34]:
trainer.train()

Epoch 1
-------------------------------
loss: 1.762536  [  600/ 4873]
loss: 2.062928  [ 1200/ 4873]
loss: 2.063450  [ 1800/ 4873]
loss: 1.911937  [ 2400/ 4873]
loss: 1.969727  [ 3000/ 4873]
loss: 1.967178  [ 3600/ 4873]
loss: 1.826272  [ 4200/ 4873]
loss: 1.972359  [ 4800/ 4873]
           accuracy  precision  recall   support
   angry     0.156     0.168     0.176    91
 disgust     0.156     0.000     0.000    91
    fear     0.156     0.174     0.505    93
   happy     0.156     0.122     0.330    88
 neutral     0.156     0.000     0.000    79
     sad     0.156     0.500     0.030    101
surprise     0.156     0.000     0.000    66
                                          609

 
     avg     0.156     0.138     0.149    

Test Error: 
 Accuracy: 15.6%, Avg loss: 1.931937 

Epoch 2
-------------------------------
loss: 1.887156  [  600/ 4873]
loss: 1.839717  [ 1200/ 4873]
loss: 2.030259  [ 1800/ 4873]
loss: 1.880090  [ 2400/ 4873]
loss: 1.997159  [ 3000/ 4873]
loss: 2.082534  [ 36

(0.3957307060755337,
 983,
 [3,
  3,
  3,
  3,
  5,
  5,
  1,
  5,
  1,
  4,
  6,
  4,
  1,
  3,
  6,
  1,
  0,
  3,
  4,
  0,
  6,
  4,
  4,
  6,
  6,
  5,
  3,
  3,
  6,
  0,
  2,
  3,
  4,
  5,
  0,
  0,
  1,
  3,
  1,
  1,
  0,
  0,
  1,
  4,
  2,
  4,
  5,
  5,
  1,
  2,
  5,
  6,
  2,
  0,
  4,
  1,
  3,
  4,
  5,
  1,
  2,
  5,
  2,
  6,
  4,
  0,
  0,
  2,
  2,
  5,
  5,
  3,
  5,
  0,
  5,
  2,
  5,
  0,
  0,
  3,
  4,
  2,
  0,
  5,
  3,
  4,
  0,
  3,
  6,
  1,
  5,
  3,
  4,
  1,
  1,
  4,
  1,
  5,
  3,
  5,
  6,
  3,
  5,
  4,
  1,
  0,
  3,
  1,
  0,
  6,
  0,
  5,
  0,
  4,
  3,
  2,
  0,
  3,
  1,
  1,
  5,
  0,
  2,
  0,
  5,
  2,
  4,
  2,
  4,
  5,
  6,
  3,
  6,
  5,
  4,
  0,
  2,
  5,
  6,
  2,
  3,
  6,
  1,
  0,
  6,
  2,
  2,
  3,
  4,
  4,
  1,
  5,
  5,
  3,
  2,
  1,
  0,
  3,
  5,
  5,
  4,
  2,
  6,
  5,
  0,
  5,
  3,
  2,
  3,
  6,
  2,
  4,
  2,
  5,
  1,
  4,
  6,
  4,
  2,
  0,
  5,
  4,
  3,
  4,
  4,
  4,
  5,
  3,
  5,
  3,
  5,
  0,
  6,
  3,
  1

In [17]:
l[0].shape

torch.Size([15884])

In [2]:
#model.load_state_dict(torch.load("../content/classifier/W2VClassifier/Nr1/emo_reco_950.pth"))
#model.from_pretrained()
#model(torch.unsqueeze(torch.tensor(trainSet[0][0]).cuda(), dim=0))